In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

In [ ]:
!echo  "{\"username\":\"sam2912\",\"key\":\"84a3fb98bc70650d712d1c5807d87eb6\"}" > ~/.kaggle/kaggle.json


In [ ]:
!cat ~/.kaggle/kaggle.json

{"username":"sam2912","key":"84a3fb98bc70650d712d1c5807d87eb6"}


In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d anudit/india-legal-cases-dataset

100% 3.32G/3.32G [00:40<00:00, 114MB/s]
100% 3.32G/3.32G [00:40<00:00, 87.3MB/s]


In [ ]:
!unzip /content/india-legal-cases-dataset.zip

Archive:  /content/india-legal-cases-dataset.zip
  inflating: merged-0-240_000.json   
  inflating: merged-1_000_000-1_160_000.json  
  inflating: merged-1_160_000-1_300_000.json  
  inflating: merged-1_300_000-1_500_000.json  
  inflating: merged-1_500_000-1_700_000.json  
  inflating: merged-1_700_000-1_850_000.json  
  inflating: merged-1_850_000-2_000_000.json  
  inflating: merged-240_000-331_000.json  
  inflating: merged-331_000-500_000.json  
  inflating: merged-500_000-620_000.json  
  inflating: merged-620_000-800_000.json  
  inflating: merged-800_000-1_000_000.json  


In [23]:
import pandas as pd
import json
import re
import multiprocessing
from multiprocessing import Pool

In [24]:
file = "/content/merged-1_000_000-1_160_000.json"
df = pd.read_json(file)

In [25]:
df.head()

,t,s,id,d,c,cl,b,a
0,"Ghasitey vs Gaon Sabha Pipara, Thru Its ... on...",Allahabad High Court,1070000,Court No. - 3\nCase :- CONSOLIDATION No. - 253...,[],0,,NaN
1,Mohd. Israr Siddiqui vs State Of U.P. And Othe...,Allahabad High Court,1070001,Court No. - 2\nCase :- WRIT - C No. - 35747 of...,[],0,,NaN
2,In Re: Sunil Kumar Bhattacharjee vs Unknown on...,Calcutta High Court,1070002,"JUDGMENT\n \n\n Basudeva Panigrahi, J. \n1....","[1450343, 1976042, 1364141, 1000421, 1976042, ...",6,,Author: B Panigrahi
3,"P.Sherly vs State Of Kerala on 15 June, 2009",Kerala High Court,1070003,\n\n \n\n \n\n \n \n IN THE HIGH COU...,[],0,,NaN
4,K.Venugopal vs The State Of Kerala on 15 Septe...,Kerala High Court,1070005,\n\n \n\n \n\n \n \n IN THE HIGH COU...,[],0,,NaN


In [26]:
def clean_string(input_string):
    # Remove newline characters and replace with a space
    cleaned_string = re.sub(r'\n', ' ', input_string)

    # Remove multiple spaces and replace with a single space
    cleaned_string = re.sub(r'\s+', ' ', cleaned_string)

    # Remove other escape characters (e.g., \t, \r)
    cleaned_string = re.sub(r'\\[ntr]', '', cleaned_string)

    # Remove multiple dashes and replace with a single dash
    cleaned_string = re.sub(r'-{2,}', '-', cleaned_string)

    # Remove multiple equals and replace with a single equals
    cleaned_string = re.sub(r'={2,}', '=', cleaned_string)

    return cleaned_string

def is_year_greater_than_2000(input_string):
    # Search for a 4-digit number in the string
    match = re.search(r'\b\d{4}\b', input_string)

    if match:
        year = int(match.group(0))
        return year >= 2000
    else:
        return False


In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

# Replace the placeholder with your Atlas connection string
uri = "<connection string>"

# Set the Stable API version when creating a new client
client = MongoClient(uri, server_api=ServerApi('1'))
                          
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

: 

In [30]:
print(f"Initial shape: {df.shape}")
for index, row in df.iterrows():
  df.loc[index, "d"] = clean_string(df.loc[index, "d"])
  if is_year_greater_than_2000(df.loc[index, "t"]):
    df.drop(index)
  # show % completion
  print((index/df.shape[0])*100)

print(f"Final shape: {df.shape}")


Streaming output truncated to the last 5000 lines.
96.33138613280103
96.33212000234839
96.33285387189574
96.33358774144308
96.33432161099043
96.33505548053778
96.33578935008514
96.33652321963247
96.33725708917983
96.33799095872718
96.33872482827452
96.33945869782188
96.34019256736921
96.34092643691658
96.34166030646392
96.34239417601127
96.34312804555863
96.34386191510596
96.34459578465332
96.34532965420067
96.34606352374801
96.34679739329538
96.34753126284271
96.34826513239007
96.34899900193741
96.34973287148476
96.35046674103211
96.35120061057947
96.35193448012681
96.35266834967416
96.35340221922151
96.35413608876885
96.35486995831621
96.35560382786356
96.35633769741091
96.35707156695825
96.3578054365056
96.35853930605296
96.35927317560031
96.36000704514765
96.36074091469501
96.36147478424235
96.36220865378971
96.36294252333705
96.3636763928844
96.36441026243176
96.3651441319791
96.36587800152645
96.36661187107379
96.36734574062115
96.3680796101685
96.36881347971584
96.3695473492632


In [31]:
sample_data_json = df.to_json()
with open(f"cleaned.json", 'w') as f:
    json.dump(sample_data_json, f)